<a href="https://colab.research.google.com/github/vishnu-kumarkeerthi/Assignment1-ML-/blob/main/Tiny_Yolo3_Deploy_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tiny yolo3 Colab Notebook

**Outline of Steps**

[keras-yolo2](https://github.com/experiencor/keras-yolo2)

[keras on google cloud ML](https://stackoverflow.com/questions/41959318/deploying-keras-models-via-google-cloud-ml)

[Yolo3 keras github](https://github.com/xiaochus/YOLOv3/blob/master/yad2k.py)

[A Practical Guide to Object Detection using the Popular YOLO Framework](https://www.analyticsvidhya.com/blog/2018/12/practical-guide-object-detection-yolo-framewor-python/)



##Reference

	@article{YOLOv3,  
	  title={YOLOv3: An Incremental Improvement},  
	  author={J Redmon, A Farhadi },
	  year={2018}
    url={https://pjreddie.com/media/files/papers/YOLOv3.pdf}


Redmon, Joseph. “Yolo Web: Real-Time Object Detection.” YOLO: Real-Time Object Detection, Joseph Redmon, 2018, pjreddie.com/darknet/yolo/.

Based on code from
Xiaochus, Larry. “YOLOv3” Github code, Xiaochus, Larry, 2018, https://github.com/xiaochus/YOLOv3.

(https://machinethink.net/blog/object-detection-with-yolo/)

    + Initialization
        + Download COCO detection data from http://cocodataset.org/#download
            + http://images.cocodataset.org/zips/train2014.zip <= train images
            + http://images.cocodataset.org/zips/val2014.zip <= validation images
            + http://images.cocodataset.org/annotations/annotations_trainval2014.zip <= train and validation annotations
        + Run this script to convert annotations in COCO format to VOC format
            + https://gist.github.com/chicham/6ed3842d0d2014987186#file-coco2pascal-py
        + Download pre-trained weights from https://pjreddie.com/darknet/yolo/
            + https://pjreddie.com/media/files/yolo.weights
        + Specify the directory of train annotations (train_annot_folder) and train images (train_image_folder)
        + Specify the directory of validation annotations (valid_annot_folder) and validation images (valid_image_folder)
        + Specity the path of pre-trained weights by setting variable *wt_path*
    + Construct equivalent network in Keras
        + Network arch from https://github.com/pjreddie/darknet/blob/master/cfg/yolo-voc.cfg
    + Load the pretrained weights
    + Perform training
    + Perform detection on an image with newly trained weights
    + Perform detection on an video with newly trained weights

# IMPORTANT!!!
Uncomment line 2 if your default runtime does not have tensorflow 2.6 installed.

In [ ]:
#!!!!IMPORTANT - RUN THIS CELL FIRST AND RESTART THE RUNTIME BEOFORE RUNNING THE OTHER CELLS
# !pip install tensorflow==2.6 #this works

# Start Here!

In [ ]:
!git clone https://github.com/xiaochus/YOLOv3.git

##Make training data directories
!mkdir yolo3_tiny

%cd yolo3_tiny

# Make image input and output directories
!mkdir images
!mkdir out

!cp ../YOLOv3/images/test/*.jpg images

!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3-tiny.cfg
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

# copy classes file to yolo3_tiny directory
!cp ../YOLOv3/data/coco_classes.txt coco_classes.txt

#Get custom fonts for image annotations
!wget -O font.zip https://fonts.google.com/download?family=Ubuntu
!unzip font.zip -d fonts

# Initialization

##Imports

In [ ]:
#numpy is a math library use to create and manipulate matricies
import numpy as np
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, Reshape, LeakyReLU, MaxPooling2D, UpSampling2D, Lambda
from keras.models import Model
from keras.layers.merge import concatenate
from keras.regularizers import l2
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from PIL import Image, ImageDraw, ImageFont

# ML Cloud serving imports
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import tag_constants, signature_constants, signature_def_utils_impl, utils
from functools import partial


In [ ]:
# print("Keras version " + keras.__version__)
print("Tensorflow version" + tf.__version__) #should be 1.15.3
!python --version #3.6.9 works

In [ ]:
LABELS = ['raccoon', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
# LABELS = ['person']

IMAGE_H, IMAGE_W = 416, 416
GRID_H,  GRID_W  = 13 , 13
BOX              = 5
CLASS            = len(LABELS)
CLASS_WEIGHTS    = np.ones(CLASS, dtype='float32')
OBJ_THRESHOLD    = 0.3#0.5
NMS_THRESHOLD    = 0.3#0.45
ANCHORS          = [10,14, 23,27, 37,58, 81,82, 135,169, 344,319]

NO_OBJECT_SCALE  = 1.0
OBJECT_SCALE     = 5.0
COORD_SCALE      = 1.0
CLASS_SCALE      = 1.0

BATCH_SIZE       = 16
WARM_UP_BATCHES  = 0
TRUE_BOX_BUFFER  = 50

In [ ]:
# wt_path = 'yolov2.weights'
# wt_path = 'yolov3.weights'
wt_path = 'yolov3-tiny.weights'
train_image_folder = './data/coco/train/'
train_annot_folder = './data/coco/trainann/'
valid_image_folder = './data/coco/val/'
valid_annot_folder = './data/coco/valann/'

# Construct the network - Model

In [ ]:
# the function to implement the orgnization layer (thanks to github.com/allanzelener/YAD2K)
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

In [ ]:
# weights_file.close()
# weights function
def setWeights(filters ,size ,count , input_shape, weights_file, doBatch):
  # ********Start weights code*************
  prev_layer_shape = K.int_shape(input_shape)

  weights_shape = (size, size, prev_layer_shape[-1], filters)
  darknet_w_shape = (filters, weights_shape[2], size, size)
  weights_size = np.product(weights_shape)

  conv_bias = np.ndarray(
    shape=(filters, ),
    dtype='float32',
    buffer=weights_file.read(filters * 4))
  count += filters

  conv_weights = []
  bn_weight_list = []

# apply batch normalization of doBatch flag is true
  if doBatch:

    bn_weights = np.ndarray(
        shape=(3, filters),
        dtype='float32',
        buffer=weights_file.read(filters * 12))
    count += 3 * filters

    # TODO: Keras BatchNormalization mistakenly refers to var
    # as std.
    bn_weight_list = [
        bn_weights[0],  # scale gamma
        conv_bias,  # shift beta
        bn_weights[1],  # running mean
        bn_weights[2]  # running var
    ]

  conv_weights = np.ndarray(
      shape=darknet_w_shape,
      dtype='float32',
      buffer=weights_file.read(weights_size * 4))
  count += weights_size

  conv_weights = np.transpose(conv_weights, [2, 3, 1, 0])

  #conv_weights = []
  if doBatch:
    conv_weights = [conv_weights] #if BatchNormalization use this
  else:
    conv_weights = [conv_weights, conv_bias] #else use this

  return conv_weights, bn_weight_list, count

# ********End weights code*************

In [ ]:
# https://github.com/xiaochus/YOLOv3/blob/master/yad2k.py
weights_file = open(wt_path, 'rb')
weights_header = np.ndarray(
        shape=(5, ), dtype='int32', buffer=weights_file.read(20))
print('Weights Header: ', weights_header)

## Build Model Layer by Layer

In [ ]:
input_image = Input(shape=(IMAGE_H, IMAGE_W, 3), name='input_1_bytes')
true_boxes  = Input(shape=(1, 1, 1, TRUE_BOX_BUFFER , 4))
weight_decay = 0.0005

#Initialize filter count variable
count = 0
# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(16, 3, count, input_image ,weights_file , True)


# Layer 1
x = Conv2D(16, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_1', use_bias=False)(input_image)
x = BatchNormalization(weights=bn_weight_list, name='norm_1')(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(32 ,3 ,count ,x ,weights_file , True)

# Layer 2
x = Conv2D(32, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_2', use_bias=False)(x)
x = BatchNormalization(weights=bn_weight_list, name='norm_2')(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(64 ,3 ,count , x,weights_file , True)

# Layer 3
x = Conv2D(64, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_3', use_bias=False)(x)
x = BatchNormalization(weights=bn_weight_list, name='norm_3')(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(128 ,3 ,count , x,weights_file , True)

# Layer 4
x = Conv2D(128, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_4', use_bias=False)(x)
x = BatchNormalization(weights=bn_weight_list, name='norm_4')(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(256 ,3 ,count ,x ,weights_file , True)

# Layer 5
x = Conv2D(256, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_5', use_bias=False)(x)
x = BatchNormalization(weights=bn_weight_list, name='norm_5')(x)
x = LeakyReLU(alpha=0.1)(x)

convd5 = x

x = MaxPooling2D(pool_size=(2, 2))(x)

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(512 ,3 ,count ,x ,weights_file , True)

# Layer 6
x = Conv2D(512, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_6', use_bias=False)(x)
x = BatchNormalization(weights=bn_weight_list, name='norm_6')(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(padding='same', pool_size=(2, 2), strides=(1,1))(x)

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(1024 ,3 ,count ,x ,weights_file , True)

# Layer 7
x = Conv2D(1024, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_7', use_bias=False)(x)
x = BatchNormalization(weights=bn_weight_list, name='norm_7')(x)
x = LeakyReLU(alpha=0.1)(x)

route7 = x

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(256 ,1 ,count ,x ,weights_file , True)

# Layer 8
x = Conv2D(256, (1,1), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_8', use_bias=False)(x)
x = BatchNormalization(weights=bn_weight_list, name='norm_8')(x)
x = LeakyReLU(alpha=0.1)(x)

convd8 = x

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(512 ,3 ,count ,x ,weights_file , True)

# Layer 9
x = Conv2D(512, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_9', use_bias=False)(x)
x = BatchNormalization(weights=bn_weight_list, name='norm_9')(x)
x = LeakyReLU(alpha=0.1)(x)

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(255 ,1 ,count ,x ,weights_file , False)

# Layer 10
x = Conv2D(255, (1,1), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same',name='conv_10', use_bias=True)(x)
n1, n2 = int(x.shape[1]), int(x.shape[2])
# x = Activation('linear')(x)

convd10 = x

# Layer 11 - yolo
n1, n2 = int(x.shape[1]), int(x.shape[2])
n3 = 3
classes = 80
n4 = (4 + 1 + classes)
yolo0 = Reshape((n1, n2, n3, n4))(x)

# Layer 12 - route
# https://github.com/AlexeyAB/darknet/issues/279#issuecomment-397248821
# Get input for next Conv layer from layer -4 index previous = Layer 7

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(128 ,1 ,count ,convd8 ,weights_file , True)

# Layer 13 - route layer
x = Conv2D(128, (1,1), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_13', use_bias=False)(convd8)
x = BatchNormalization(weights=bn_weight_list, name='norm_13')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 14 - upsample
# x = ZeroPadding2D(((1, 0), (1, 0)))(convd10) # Adjust padding model for darknet.
x = UpSampling2D(size=(2, 2))(x)

# Layer 15 - route
# https://github.com/AlexeyAB/darknet/issues/279#issuecomment-397248821
# Get input for next Conv layer from layer -1 index previous = Layer 14 and Layer 8
# skip_connection = Lambda(space_to_depth_x2)(x)
# x = concatenate([route8, skip_connection])
x = concatenate([convd5, x])

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(256 ,3 ,count ,x ,weights_file , True)

# Layer 16
x = Conv2D(256, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_16', use_bias=False)(x)
x = BatchNormalization(weights=bn_weight_list, name='norm_16')(x)
x = LeakyReLU(alpha=0.1)(x)

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(255 ,1 ,count ,x ,weights_file , False)

# Layer 17
x = Conv2D(255, (1,1), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_17', use_bias=True)(x)

# Layer 18 - yolo
n1, n2 = int(x.shape[1]), int(x.shape[2])
yolo1 = Reshape((n1, n2, 3, 85))(x)

# Create and save model.
model = Model(inputs=[input_image],  outputs=[yolo0, yolo1])

# Export to hd5 file
model.save('{}'.format('yolotest.h5'))

In [ ]:
model.summary()

In [ ]:
# Check to see if all weights have been read.
remaining_weights = len(weights_file.read()) / 4

print('Read {} of {} from Darknet weights.'.format(count, count +
remaining_weights))

weights_file.close()

##Perform detection on image

[Based on code from xiaochus github repo](https://github.com/xiaochus/YOLOv3)

###Load yolo model from file

In [ ]:
yolo = load_model('yolotest.h5')
# yolo = model

###Image processing functions
[Based on code from xiaochus github repo](https://github.com/xiaochus/YOLOv3)

In [ ]:
# obj_threshold
# t1 = 0.6
t1 = 0.1
# nms_threshold
t2 = 0.5

def sigmoid(x):
        """sigmoid.

        # Arguments
            x: Tensor.

        # Returns
            numpy ndarray.
        """
        return 1 / (1 + np.exp(-x))

def process_feats(out, anchors, mask):
    """process output features.

    # Arguments
        out: Tensor (N, N, 3, 4 + 1 +80), output feature map of yolo.
        anchors: List, anchors for box.
        mask: List, mask for anchors.

    # Returns
        boxes: ndarray (N, N, 3, 4), x,y,w,h for per box.
        box_confidence: ndarray (N, N, 3, 1), confidence for per box.
        box_class_probs: ndarray (N, N, 3, 80), class probs for per box.
    """
    grid_h, grid_w, num_boxes = map(int, out.shape[1: 4])

    anchors = [anchors[i] for i in mask]
    anchors_tensor = np.array(anchors).reshape(1, 1, len(anchors), 2)

    # Reshape to batch, height, width, num_anchors, box_params.
    out = out[0]
    box_xy = sigmoid(out[..., :2])
    box_wh = np.exp(out[..., 2:4])
    box_wh = box_wh * anchors_tensor

    box_confidence = sigmoid(out[..., 4])
    box_confidence = np.expand_dims(box_confidence, axis=-1)
    box_class_probs = sigmoid(out[..., 5:])

    col = np.tile(np.arange(0, grid_w), grid_w).reshape(-1, grid_w)
    row = np.tile(np.arange(0, grid_h).reshape(-1, 1), grid_h)

    col = col.reshape(grid_h, grid_w, 1, 1).repeat(3, axis=-2)
    row = row.reshape(grid_h, grid_w, 1, 1).repeat(3, axis=-2)
    grid = np.concatenate((col, row), axis=-1)

    box_xy += grid
    box_xy /= (grid_w, grid_h)
    box_wh /= (416, 416)
    box_xy -= (box_wh / 2.)
    boxes = np.concatenate((box_xy, box_wh), axis=-1)

    return boxes, box_confidence, box_class_probs

def nms_boxes(boxes, scores):
    """Suppress non-maximal boxes.

    # Arguments
        boxes: ndarray, boxes of objects.
        scores: ndarray, scores of objects.

    # Returns
        keep: ndarray, index of effective boxes.
    """
    x = boxes[:, 0]
    y = boxes[:, 1]
    w = boxes[:, 2]
    h = boxes[:, 3]

    areas = w * h
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)

        xx1 = np.maximum(x[i], x[order[1:]])
        yy1 = np.maximum(y[i], y[order[1:]])
        xx2 = np.minimum(x[i] + w[i], x[order[1:]] + w[order[1:]])
        yy2 = np.minimum(y[i] + h[i], y[order[1:]] + h[order[1:]])

        w1 = np.maximum(0.0, xx2 - xx1 + 1)
        h1 = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w1 * h1

        ovr = inter / (areas[i] + areas[order[1:]] - inter)
        inds = np.where(ovr <= t2)[0]
        order = order[inds + 1]

    keep = np.array(keep)

    return keep

def filter_boxes(boxes, box_confidences, box_class_probs):
    """Filter boxes with object threshold.

    # Arguments
        boxes: ndarray, boxes of objects.
        box_confidences: ndarray, confidences of objects.
        box_class_probs: ndarray, class_probs of objects.

    # Returns
        boxes: ndarray, filtered boxes.
        classes: ndarray, classes for boxes.
        scores: ndarray, scores for boxes.
    """
    box_scores = box_confidences * box_class_probs
    box_classes = np.argmax(box_scores, axis=-1)
    box_class_scores = np.max(box_scores, axis=-1)
    pos = np.where(box_class_scores >= t1)

    boxes = boxes[pos]
    classes = box_classes[pos]
    scores = box_class_scores[pos]

    return boxes, classes, scores

def yolo_out(outs, shape):
    """Process output of yolo base net.

    # Argument:
        outs: output of yolo base net.
        shape: shape of original image.

    # Returns:
        boxes: ndarray, boxes of objects.
        classes: ndarray, classes of objects.
        scores: ndarray, scores of objects.
    """
    masks = [[6, 7, 8], [3, 4, 5], [0, 1, 2]]
    anchors = [[10, 13], [16, 30], [33, 23], [30, 61], [62, 45],
               [59, 119], [116, 90], [156, 198], [373, 326]]

    boxes, classes, scores = [], [], []

    for out, mask in zip(outs, masks):
        b, c, s = process_feats(out, anchors, mask)
        b, c, s = filter_boxes(b, c, s)
        boxes.append(b)
        classes.append(c)
        scores.append(s)

    boxes = np.concatenate(boxes)
    classes = np.concatenate(classes)
    scores = np.concatenate(scores)

    # Scale boxes back to original image shape.
    width, height = shape[0], shape[1]
    image_dims = [width, height, width, height]
    boxes = boxes * image_dims

    nboxes, nclasses, nscores = [], [], []
    for c in set(classes):
        inds = np.where(classes == c)
        b = boxes[inds]
        c = classes[inds]
        s = scores[inds]

        keep = nms_boxes(b, s)

        nboxes.append(b[keep])
        nclasses.append(c[keep])
        nscores.append(s[keep])

    if not nclasses and not nscores:
        return None, None, None

    boxes = np.concatenate(nboxes)
    classes = np.concatenate(nclasses)
    scores = np.concatenate(nscores)

    return boxes, classes, scores

def draw(image, boxes, scores, classes, all_classes):

  """Draw the boxes on the image.
    # Argument:
        image: original image.
        boxes: ndarray, boxes of objects.
        classes: ndarray, classes of objects.
        scores: ndarray, scores of objects.
        all_classes: all classes name.
  """
#uncomment to use a custom font uploaded to this notebook directory
  fnt =ImageFont.truetype('./fonts/Ubuntu-Bold.ttf', 18)

  for box, score, cl in zip(boxes, scores, classes):

      x, y, w, h = box

      top = max(0, np.floor(x + 0.5).astype(int))
      left = max(0, np.floor(y + 0.5).astype(int))
      right = min(image.size[0], np.floor(x + w + 0.5).astype(int))
      bottom = min(image.size[1], np.floor(y + h + 0.5).astype(int))

      draw = ImageDraw.Draw(image)
      draw.rectangle([(top, left), (right, bottom)], outline=(0,255,0))#blue rectangle
      draw.text((top, left - 6), '{0} {1:.2f}'.format(all_classes[cl], score), font=fnt, fill=(0, 0, 255))
#      draw.text((top, left - 6), '{0} {1:.2f}'.format(all_classes[cl], score), fill=(0, 0, 255))
#         draw.rectangle([(left, top), (right, bottom)], outline=(0,0,255))#blue rectangle
      print('class: {0}, score: {1:.2f}'.format(all_classes[cl], score))
      print('box coordinate x,y,w,h: {0}'.format(box))

  print()



###Load Image
Resize, reduce and expand image.

In [ ]:
# !ls -l
%cd yolo3_tiny

In [ ]:
#Pick one image to process by uncommenting it
input_image_name = 'person.jpg'
# input_image_name = 'giraffe.jpg'
# input_image_name = 'dog.jpg'
# input_image_name = 'eagle.jpg'
# input_image_name = 'horses.jpg'
# input_image_name = 'toysoldiers.jpg'
# input_image_name = 'toycar1.jpg'

size = (416, 416)
image_src = Image.open("images/" + input_image_name)
orig_size = image_src.size
image_thumb = image_src.resize(size, Image.BICUBIC)
image_thumb.save("out/thumb_" + input_image_name, "JPEG") #save thumbnail version
image = np.array(image_thumb, dtype='float32')
image /= 255.
image = np.expand_dims(image, axis=0)

###Get prediction output

In [ ]:
# Raw Prediction Output
output = yolo.predict(image)


In [ ]:
with open('coco_classes.txt') as f:
    class_names = f.readlines()
all_classes = [c.strip() for c in class_names]

# Processed Output
image_thumb = Image.open("out/thumb_" + input_image_name) #open thumbnail image
thumb_size = image_thumb.size
boxes, classes, scores = yolo_out(output, thumb_size)  #process thumbnail image
#boxes, classes, scores = yolo_out(output, orig_size)
if boxes is not None:
  draw(image_thumb, boxes, scores, classes, all_classes) #annotate thumbnail image
  #draw(image_src, boxes, scores, classes, all_classes)

# Display processed image output
# image_src.show()
#image_src.save("out/" + input_image_name, "JPEG")
image_thumb.save("out/" + input_image_name, "JPEG")

In [ ]:
%cd ..
!pwd

# Prepare Model for Saving

In [ ]:
# Go up one directory level
%cd ..
# Save model for deployment
model.save("tinyYolo3")

In [ ]:
# Zip saved XOR model directory for download to local file system
!zip -r tinyYolo3.zip tinyYolo3

In [ ]:
# Get model input signature
model.input

## Attempt to perform cloud object detection by deploying our model to Vertex AI.
### Using the python request library
This does not work because it returns the following error.

`{'error': {'code': 400, 'message': 'The request size (2769005 bytes) exceeds 1.500MB limit.', 'status': 'FAILED_PRECONDITION'}}`

Refer to our other [Black Magic AI Videos](https://www.youtube.com/channel/UCR7R8zwZjqY3c-a3l4xw1mQ) for other options.

In [ ]:
# Command to Authenticate with your Google account.
!gcloud auth login

In [ ]:
# Display the BEARER_TOKEN then copy and paste into the BEARER_TOKEN variable value in the next two cells
!gcloud auth print-access-token

In [ ]:
# Ref:
# https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models

import requests
import base64

PROJECT_ID='<INSERT_PROJECT_ID_HERE>'
ENDPOINT_ID='<INSERT_ENDPOINT_ID_HERE>'
BEARER_TOKEN = '<INSERT_BEARER_TOKEN_HERE>'
url = 'https://us-central1-aiplatform.googleapis.com/v1/projects/{}/locations/us-central1/endpoints/{}:predict'.format(PROJECT_ID, ENDPOINT_ID)
bearer = 'Bearer {}'.format(BEARER_TOKEN)

response = requests.post(url,
                  headers={'Content-Type': 'application/json',
                           'Authorization': bearer},
                  json={"instances": [{"b64": base64.b64encode(image).decode()}]}
                  )
print(response.json())
# print(response.json()['predictions'][0])

# Task
Modernize the Tiny YOLOv3 object detection pipeline by migrating the codebase to TensorFlow 2.x and `tensorflow.keras`. This includes refactoring the model architecture and weight loading from "https://pjreddie.com/media/files/yolov3-tiny.weights" using the modern Keras functional API, updating image processing and Non-Maximum Suppression (NMS) functions for compatibility with recent NumPy and TensorFlow versions, and ensuring the model is exported in the SavedModel format. Use the configuration from "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3-tiny.cfg" and class labels from "https://github.com/xiaochus/YOLOv3.git" to demonstrate the updated object detection pipeline.

## Update Environment and Imports

### Subtask:
Prepare the notebook for TensorFlow 2.x and migrate all 'keras' imports to 'tensorflow.keras' to align with modern standards.


**Reasoning**:
I will provide the necessary code to refactor the imports from the standalone 'keras' library to 'tensorflow.keras' and verify the TensorFlow 2.x environment as per the subtask instructions.



In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, Reshape, LeakyReLU, MaxPooling2D, UpSampling2D, Lambda, concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from PIL import Image, ImageDraw, ImageFont
from functools import partial

# Verify TensorFlow 2.x
print(f"TensorFlow version: {tf.__version__}")
if not tf.__version__.startswith('2.'):
    print("Warning: This notebook is designed for TensorFlow 2.x.")
else:
    print("Environment verified: Running TensorFlow 2.x.")

TensorFlow version: 2.19.0
Environment verified: Running TensorFlow 2.x.


## Refactor Weight Loading and Model Construction

### Subtask:
Update the weight-loading logic and rebuild the Tiny YOLOv3 architecture using the modern Keras functional API.


**Reasoning**:
I will initialize the global variables for the model dimensions, anchors, and class labels, and then download the required weight and configuration files from the official Darknet sources.



In [2]:
import os

# 1. Initialize Global Variables
IMAGE_H, IMAGE_W = 416, 416
# Tiny YOLOv3 Anchors
ANCHORS = [10, 14, 23, 27, 37, 58, 81, 82, 135, 169, 344, 319]
# Load COCO classes if they exist, else use a placeholder or download
CLASSES_PATH = 'coco_classes.txt'

if not os.path.exists(CLASSES_PATH):
    print("Downloading coco_classes.txt...")
    !wget -q https://raw.githubusercontent.com/xiaochus/YOLOv3/master/data/coco_classes.txt

with open(CLASSES_PATH, 'r') as f:
    LABELS = [c.strip() for c in f.readlines()]

print(f"Loaded {len(LABELS)} classes.")

# 2. Download Weights and Config Files
WEIGHTS_FILE = 'yolov3-tiny.weights'
CONFIG_FILE = 'yolov3-tiny.cfg'

if not os.path.exists(WEIGHTS_FILE):
    print(f"Downloading {WEIGHTS_FILE}...")
    !wget -q https://pjreddie.com/media/files/yolov3-tiny.weights

if not os.path.exists(CONFIG_FILE):
    print(f"Downloading {CONFIG_FILE}...")
    !wget -q https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3-tiny.cfg

print("Environment and files initialized.")

Loaded 80 classes.
Environment and files initialized.


**Reasoning**:
I will define the WeightReader class to parse the binary Darknet weights and implement the Tiny YOLOv3 architecture using the TensorFlow Keras functional API. Then, I will instantiate the model, load the pre-trained parameters, and verify the model summary.



In [3]:
class WeightReader:
    def __init__(self, weight_file):
        with open(weight_file, 'rb') as w_f:
            # Read header
            major, = np.fromfile(w_f, dtype='int32', count=1)
            minor, = np.fromfile(w_f, dtype='int32', count=1)
            revision, = np.fromfile(w_f, dtype='int32', count=1)
            if (major * 10 + minor) >= 2 and major < 1000:
                w_f.read(8)
            else:
                w_f.read(4)
            binary = w_f.read()
        self.offset = 0
        self.all_weights = np.frombuffer(binary, dtype='float32')

    def read_bytes(self, size):
        self.offset = self.offset + size
        return self.all_weights[self.offset - size : self.offset]

def _conv_block(inputs, filters, size, stride, batch_normalize, name):
    padding = 'same'
    # Standard Tiny YOLOv3 doesn't use the 'valid' padding trick for these strides
    x = Conv2D(filters, size, strides=stride, padding=padding,
               use_bias=not batch_normalize, kernel_regularizer=l2(0.0005), name=name)(inputs)
    if batch_normalize:
        x = BatchNormalization(epsilon=1e-3, name=name + '_bn')(x)
        x = LeakyReLU(alpha=0.1, name=name + '_leaky')(x)
    return x

def build_tiny_yolo(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    # Layer 0-1
    x = _conv_block(inputs, 16, 3, 1, True, 'conv_0')
    x = MaxPooling2D(2, 2, padding='same')(x)
    # Layer 2-3
    x = _conv_block(x, 32, 3, 1, True, 'conv_1')
    x = MaxPooling2D(2, 2, padding='same')(x)
    # Layer 4-5
    x = _conv_block(x, 64, 3, 1, True, 'conv_2')
    x = MaxPooling2D(2, 2, padding='same')(x)
    # Layer 6-7
    x = _conv_block(x, 128, 3, 1, True, 'conv_3')
    x = MaxPooling2D(2, 2, padding='same')(x)
    # Layer 8
    layer_8 = _conv_block(x, 256, 3, 1, True, 'conv_4')
    # Layer 9
    x = MaxPooling2D(2, 2, padding='same')(layer_8)
    # Layer 10-11
    x = _conv_block(x, 512, 3, 1, True, 'conv_5')
    x = MaxPooling2D(2, 1, padding='same')(x)
    # Layer 12-14
    x = _conv_block(x, 1024, 3, 1, True, 'conv_6')
    layer_13 = _conv_block(x, 256, 1, 1, True, 'conv_7')
    x = _conv_block(layer_13, 512, 3, 1, True, 'conv_8')
    # Output 1
    yolo_1 = _conv_block(x, 3 * (5 + num_classes), 1, 1, False, 'conv_9')
    # Route for Branch 2
    x = _conv_block(layer_13, 128, 1, 1, True, 'conv_10')
    x = UpSampling2D(2)(x)
    x = concatenate([x, layer_8])
    x = _conv_block(x, 256, 3, 1, True, 'conv_11')
    # Output 2
    yolo_2 = _conv_block(x, 3 * (5 + num_classes), 1, 1, False, 'conv_12')
    return Model(inputs, [yolo_1, yolo_2])

# Instantiate model
model = build_tiny_yolo((IMAGE_H, IMAGE_W, 3), len(LABELS))

# Load Weights
reader = WeightReader(WEIGHTS_FILE)
conv_layers = [l for l in model.layers if isinstance(l, Conv2D)]

for i in range(len(conv_layers)):
    conv_layer = conv_layers[i]
    # Output layers in Tiny YOLOv3 (conv_9 and conv_12) don't have BN
    bn_layer = None
    try:
        bn_layer = model.get_layer(conv_layer.name + '_bn')
    except:
        pass

    if bn_layer:
        size = np.prod(bn_layer.get_weights()[0].shape)
        beta = reader.read_bytes(size)
        gamma = reader.read_bytes(size)
        mean = reader.read_bytes(size)
        var = reader.read_bytes(size)
        bn_layer.set_weights([gamma, beta, mean, var])
    else:
        bias = reader.read_bytes(conv_layer.filters)
        conv_layer.set_weights([conv_layer.get_weights()[0], bias])

    # Load Conv weights
    w_shape = conv_layer.get_weights()[0].shape
    # Darknet: (out, in, h, w) -> Keras: (h, w, in, out)
    size = np.prod(w_shape)
    weights = reader.read_bytes(size)
    weights = weights.reshape((w_shape[3], w_shape[2], w_shape[0], w_shape[1]))
    weights = np.transpose(weights, (2, 3, 1, 0))

    if bn_layer:
        conv_layer.set_weights([weights])
    else:
        conv_layer.set_weights([weights, conv_layer.get_weights()[1]])

print(f"Weights fully read: {reader.offset == len(reader.all_weights)}")
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Weights fully read: True


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 416, 416,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_0 (Conv2D)     │ (None, 416, 416,  │        432 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_0_bn           │ (None, 416, 416,  │         64 │ conv_0[0][0]      │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_0_leaky        │ (None, 416, 416,  │          0 │ conv_0_bn[0][0]   │
│ (LeakyReLU)         │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 208, 208,  │          0 │ conv_0_leaky[0][… │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_1 (Conv2D)     │ (None, 208, 208,  │      4,608 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_1_bn           │ (None, 208, 208,  │        128 │ conv_1[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_1_leaky        │ (None, 208, 208,  │          0 │ conv_1_bn[0][0]   │
│ (LeakyReLU)         │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 104, 104,  │          0 │ conv_1_leaky[0][… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_2 (Conv2D)     │ (None, 104, 104,  │     18,432 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_2_bn           │ (None, 104, 104,  │        256 │ conv_2[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_2_leaky        │ (None, 104, 104,  │          0 │ conv_2_bn[0][0]   │
│ (LeakyReLU)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 52, 52,    │          0 │ conv_2_leaky[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_3 (Conv2D)     │ (None, 52, 52,    │     73,728 │ max_pooling2d_2[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_3_bn           │ (None, 52, 52,    │        512 │ conv_3[0][0]      │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_3_leaky        │ (None, 52, 52,    │          0 │ conv_3_bn[0][0]   │
│ (LeakyReLU)         │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 26, 26,    │          0 │ conv_3_leaky[0][

 Total params: 8,858,734 (33.79 MB)

 Trainable params: 8,852,366 (33.77 MB)

 Non-trainable params: 6,368 (24.88 KB)

# Task
Modernize the Tiny YOLOv3 object detection pipeline by refactoring image processing, NMS, and box filtering logic for compatibility with TensorFlow 2.x, and export the model to SavedModel format using weights from "https://pjreddie.com/media/files/yolov3-tiny.weights" and configuration from "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3-tiny.cfg".

## Modernize Image Processing and Prediction Logic

### Subtask:
Refactor the NMS, box filtering, and image annotation functions for compatibility and performance with current NumPy and TensorFlow versions.


**Reasoning**:
I will implement the modernized image processing and prediction logic, including sigmoid transformation, output decoding, NMS, filtering, and visualization utilities, ensuring compatibility with TensorFlow 2.x and current NumPy versions.



In [4]:
def _sigmoid(x):
    return 1. / (1. + np.exp(-x))

def decode_netout(netout, anchors, obj_thresh, net_h, net_w):
    grid_h, grid_w = netout.shape[:2]
    nb_box = 3
    netout = netout.reshape((grid_h, grid_w, nb_box, -1))
    nb_class = netout.shape[-1] - 5

    boxes = []

    netout[..., :2]  = _sigmoid(netout[..., :2])
    netout[..., 4:]  = _sigmoid(netout[..., 4:])
    netout[..., 5:]  = netout[..., 4][..., np.newaxis] * netout[..., 5:]
    netout[..., 5:] *= netout[..., 5:] > obj_thresh

    for i in range(grid_h*grid_w):
        row = i // grid_w
        col = i % grid_w

        for b in range(nb_box):
            # 4th element is objectness score
            objectness = netout[row, col, b, 4]

            if(objectness <= obj_thresh): continue

            # first 4 elements are x, y, w, and h
            x, y, w, h = netout[row, col, b, :4]

            x = (col + x) / grid_w # center position, unit: image width
            y = (row + y) / grid_h # center position, unit: image height
            w = anchors[2 * b + 0] * np.exp(w) / net_w # unit: image width
            h = anchors[2 * b + 1] * np.exp(h) / net_h # unit: image height

            # last elements are class probabilities
            classes = netout[row, col, b, 5:]

            box = [x-w/2, y-h/2, x+w/2, y+h/2, objectness, classes]
            boxes.append(box)

    return boxes

def correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w):
    if (float(net_w)/image_w) < (float(net_h)/image_h):
        new_w = net_w
        new_h = (image_h*net_w)/image_w
    else:
        new_h = net_w
        new_w = (image_w*net_h)/image_h

    for i in range(len(boxes)):
        x_offset, x_scale = (net_w - new_w)/2./net_w, float(new_w)/net_w
        y_offset, y_scale = (net_h - new_h)/2./net_h, float(new_h)/net_h

        boxes[i][0] = (boxes[i][0] - x_offset) / x_scale * image_w
        boxes[i][2] = (boxes[i][2] - x_offset) / x_scale * image_w
        boxes[i][1] = (boxes[i][1] - y_offset) / y_scale * image_h
        boxes[i][3] = (boxes[i][3] - y_offset) / y_scale * image_h

def _interval_overlap(interval_a, interval_b):
    x1, x2 = interval_a
    x3, x4 = interval_b

    if x3 < x1:
        if x4 < x1: return 0
        else: return min(x2,x4) - x1
    else:
        if x2 < x3: return 0
        else: return min(x2,x4) - x3

def bbox_iou(box1, box2):
    intersect_w = _interval_overlap([box1[0], box1[2]], [box2[0], box2[2]])
    intersect_h = _interval_overlap([box1[1], box1[3]], [box2[1], box2[3]])

    intersect = intersect_w * intersect_h

    w1, h1 = box1[2]-box1[0], box1[3]-box1[1]
    w2, h2 = box2[2]-box2[0], box2[3]-box2[1]

    union = w1*h1 + w2*h2 - intersect

    return float(intersect) / union

def do_nms(boxes, nms_thresh):
    if len(boxes) > 0:
        nb_class = len(boxes[0][5])
    else:
        return

    for c in range(nb_class):
        sorted_indices = np.argsort([-box[5][c] for box in boxes])

        for i in range(len(sorted_indices)):
            index_i = sorted_indices[i]

            if boxes[index_i][5][c] == 0: continue

            for j in range(i+1, len(sorted_indices)):
                index_j = sorted_indices[j]

                if bbox_iou(boxes[index_i], boxes[index_j]) >= nms_thresh:
                    boxes[index_j][5][c] = 0

def draw_boxes(image, boxes, labels, obj_thresh):
    draw = ImageDraw.Draw(image)
    try:
        font = ImageFont.truetype("fonts/Ubuntu-Bold.ttf", 15)
    except IOError:
        font = ImageFont.load_default()

    for box in boxes:
        label_str = ""
        label = -1

        for i in range(len(labels)):
            if box[5][i] > obj_thresh:
                label_str += labels[i]
                label = i
                print(f"{labels[i]}: {box[5][i]*100:.2f}%")

        if label >= 0:
            draw.rectangle([box[0], box[1], box[2], box[3]], outline="red", width=3)
            draw.text((box[0], box[1] - 15), f"{label_str} {box[5][label]:.2f}", fill="red", font=font)
    return image

def preprocess_image(image_path, net_h, net_w):
    image = Image.open(image_path)
    image_w, image_h = image.size
    new_image = image.resize((net_w, net_h))

    x_batch = np.array(new_image, dtype='float32')
    x_batch /= 255.
    x_batch = np.expand_dims(x_batch, 0)

    return x_batch, image_w, image_h

print("Modernized image processing and prediction functions defined.")

Modernized image processing and prediction functions defined.
